## Feature Selection

In [1]:
import findspark
import pandas as pd
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkConf

# for shared metastore (shared across all users)
spark = SparkSession.builder.appName("List available databases and tables").config("hive.metastore.uris", "thrift://bialobog:9083", conf=SparkConf()).getOrCreate() \

# for local metastore (your private, invidivual database) add the following config to spark session

spark.catalog.listDatabases()

[Database(name='2022_10_22', catalog='spark_catalog', description='FactSet data version for the day', locationUri='hdfs://bialobog.cs.ucl.ac.uk:8020/user/hive/warehouse'),
 Database(name='2023_04_01', catalog='spark_catalog', description='FactSet data version for the day', locationUri='hdfs://bialobog.cs.ucl.ac.uk:8020/user/hive/warehouse'),
 Database(name='default', catalog='spark_catalog', description='Default Hive database', locationUri='hdfs://bialobog.cs.ucl.ac.uk:8020/user/hive/warehouse')]

In [2]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import when

spark.sql("USE 2023_04_01")


DataFrame[]

### Investigating metrics that changed the most before and after implosions

In [8]:

def get_all_stocks_df():
    query = f"""SELECT s.ticker_region, s.fsym_id FROM sym_ticker_region s 
                LEFT JOIN FF_SEC_COVERAGE c ON c.fsym_id = s.fsym_id
                LEFT JOIN sym_coverage sc ON sc.fsym_id = s.fsym_id
                WHERE s.ticker_region LIKE "%-US" AND s.ticker_region NOT LIKE '%.%' AND c.CURRENCY = "USD"
                AND (sc.fref_listing_exchange = "NAS" OR sc.fref_listing_exchange = "NYS")"""
    df = spark.sql(query)
    df = df.withColumn("ticker_region", regexp_replace("ticker_region", "-US$", ""))
    return df


def get_not_null_cols(df, table):
    df=spark.createDataFrame(df)
    df.createOrReplaceTempView("temp_table")
    query1 = f"""SELECT t.fsym_id, a.*
                FROM temp_table t
                LEFT JOIN {table} a ON t.fsym_id = a.fsym_id
                WHERE a.date >= '2000-01-01'
                ORDER BY t.fsym_id, a.date
            """
    q_df = spark.sql(query1)
    remove_cols = ['fsym_id', 'date', 'adjdate', 'currency']
    q_df = q_df.drop(*remove_cols)
    q_df = ps.DataFrame(q_df)
    null_pcts = q_df.isnull().sum()/len(q_df)
    cols = null_pcts[null_pcts <= 0.2].index.tolist()
    return cols


In [4]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import when, lit, col
import pyspark.pandas as ps
import numpy as np
from scipy.stats import zscore
import matplotlib.pyplot as plt
import csv


def pct_change_df(df, big_string, table):
    df=spark.createDataFrame(df)
    df.createOrReplaceTempView("temp_table")
    
    query1 = f"""
                SELECT t.fsym_id, t.Implosion_Start_Date, b.date, {big_string}
                FROM temp_table t  
                LEFT JOIN sym_ticker_region s ON s.fsym_id = t.fsym_id
                LEFT JOIN {table} a ON s.fsym_id = a.fsym_id AND  YEAR(a.date) = YEAR(t.Implosion_Start_Date)
                LEFT JOIN {table} b ON s.fsym_id = b.fsym_id AND  YEAR(b.date) = YEAR(t.Implosion_Start_Date)-1
                ORDER BY t.fsym_id
            """
    df1 = spark.sql(query1)
    #print(df1.show())
    df1 = df1.toPandas()
    df1 = df1.drop(['fsym_id','Implosion_Start_Date','date'], axis=1)
    
    def remove_outliers(column):
        Q1 = column.quantile(0.25)
        Q3 = column.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        return column[(column >= lower_bound) & (column <= upper_bound)]



    df1 = df1.abs()
    null_percentage = df1.isnull().mean()
    columns_to_keep = null_percentage[null_percentage <= 0.2].index
    df_nulls_removed = df1[columns_to_keep]
    print("Columns kept: ", len(columns_to_keep)/len(df1.columns))
    
    df_no_outliers = df_nulls_removed.apply(remove_outliers)

    
    column_means_no_outliers = df_no_outliers.mean()
    #column_means_no_outliers = column_means_no_outliers.dropna()
    column_means_no_outliers = column_means_no_outliers.sort_values()
    feats = column_means_no_outliers.tail(10)

    print("Largest averages of differences between previous year and implosion year: ",feats)
    return feats.index.tolist()
    
def avg_change_df(df, big_string, table):
    df=spark.createDataFrame(df)
    df.createOrReplaceTempView("temp_table")
    
    query1 = f"""
                SELECT t.fsym_id, {big_string}
                FROM temp_table t  
                LEFT JOIN sym_ticker_region s ON s.fsym_id = t.fsym_id
                LEFT JOIN {table} a ON s.fsym_id = a.fsym_id AND  YEAR(a.date) > YEAR(t.Implosion_Start_Date)
                LEFT JOIN {table} b ON s.fsym_id = b.fsym_id AND  YEAR(b.date) < YEAR(t.Implosion_Start_Date)
                GROUP BY t.fsym_id
                ORDER BY t.fsym_id
            """
    df1 = spark.sql(query1)
    df1 = df1.toPandas()
    df1 = df1.drop(['fsym_id'], axis=1)
    
    def remove_outliers(column):
        Q1 = column.quantile(0.25)
        Q3 = column.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        return column[(column >= lower_bound) & (column <= upper_bound)]


    df1 = df1.abs()
    null_percentage = df1.isnull().mean()
    columns_to_keep = null_percentage[null_percentage <= 0.2].index
    df_nulls_removed = df1[columns_to_keep]
    print("Columns kept: ", len(columns_to_keep)/len(df1.columns))
    
    df_no_outliers = df_nulls_removed.apply(remove_outliers)
    
    column_means_no_outliers = df_no_outliers.mean()
    #column_means_no_outliers = column_means_no_outliers.dropna()
    column_means_no_outliers = column_means_no_outliers.sort_values()
    feats = column_means_no_outliers.tail(10)
    print("Largest averages of differences in average before and after implosion date: ", feats)
    return feats.index.tolist()


def get_metric_changes(filename, table):
    df = pd.read_csv(filename, index_col=False)
    df = df[df['Implosion_Start_Date'].notnull()]
    df['Implosion_Start_Date'] = pd.to_datetime(df['Implosion_Start_Date']).dt.date
    df['Implosion_End_Date'] = pd.to_datetime(df['Implosion_End_Date']).dt.date
    cols = get_not_null_cols(df, table)
    print("cols not null:", cols)
    result_string = ', '.join('(a.' + item + '-b.' + item +')/b.'+item + ' AS ' + item for item in cols)
    feats1 = pct_change_df(df, result_string, table) #change 1 year before
    print(feats1)
    
    result_string2 = ', '.join('(MEAN(a.' + item + ')-MEAN(b.' + item +'))/MEAN(b.'+item + ') AS ' + item for item in cols)
    feats2 = avg_change_df(df, result_string2, table)
    print("Final features: ", list(set(feats1+feats2)))
    csv_file_path = "features.csv"
    data_list = [list(set(feats1+feats2))]
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        for row in data_list:
            writer.writerow(row)


#get_metric_changes('imploded_stocks_price.csv', 'FF_ADVANCED_DER_AF')


/opt/spark/python/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


### Correlations with Price

In [5]:
from pyspark.sql.functions import when, lit, col

def get_fund_data(imp_df):
    imp_df.createOrReplaceTempView("temp_table")
    query2 = f"""SELECT t.fsym_id, p.p_date, p.p_price, splits.p_split_date, splits.p_split_factor, 
                divs.p_divs_exdate, divs.p_divs_s_pd, divs.p_divs_pd, ms.p_com_shs_out 
                FROM temp_table t
                LEFT JOIN sym_ticker_region s ON s.fsym_id = t.fsym_id
                LEFT JOIN FF_SEC_COVERAGE c ON c.fsym_id = s.fsym_id
                LEFT JOIN sym_coverage sc ON sc.fsym_id = s.fsym_id
                INNER JOIN fp_basic_prices p ON p.fsym_id = sc.fsym_regional_id
                LEFT JOIN fp_basic_dividends divs ON divs.p_divs_exdate = p.p_date AND p.fsym_id = divs.fsym_id
                LEFT JOIN fp_basic_splits AS splits ON splits.p_split_date = p.p_date AND p.fsym_id = splits.fsym_id
                LEFT JOIN (SELECT sf.fsym_id, mp.price_date, sf.p_com_shs_out, sf.p_date AS shares_hist_date
                                        FROM fp_basic_shares_hist AS sf
                                        JOIN (SELECT p2.fsym_id, p2.p_date AS price_date, max(h.p_date) AS max_shares_hist_date
                                                FROM fp_basic_prices AS p2
                                                JOIN fp_basic_shares_hist AS h ON h.p_date <= p2.p_date AND p2.fsym_id = h.fsym_id
                                                GROUP BY p2.fsym_id, p2.p_date)
                                        mp ON mp.fsym_id = sf.fsym_id AND mp.max_shares_hist_date = sf.p_date)
                            ms ON ms.fsym_id = p.fsym_id AND ms.price_date = p.p_date
                WHERE p.p_date >= '2000-01-01'
                ORDER BY s.fsym_id, p.p_date
                """
    adj = spark.sql(query2)
    imp_df = imp_df.toPandas()
    adj.show()
    adj = adj.withColumn("temp_cum_split_factor", when(adj.p_date==adj.p_split_date, F.lit(adj.p_split_factor)).otherwise(F.lit(1.0)))
    adj = adj.withColumn("cum_split_factor", F.lit(0.0))

    window_spec = Window.partitionBy('fsym_id').orderBy(F.desc('p_date'))

    adj = adj.withColumn('cum_split_factor_no_lag', 
                        F.exp(F.sum(F.log('temp_cum_split_factor')).over(window_spec)))

    adj = adj.withColumn('cum_split_factor', 
                        F.when(F.row_number().over(window_spec) == 1, 1.0)
                        .otherwise(F.lag('cum_split_factor_no_lag', default=1.0).over(window_spec)))
    
    adj = adj.withColumn('split_adj_price', adj.p_price * adj.cum_split_factor)
    
    adj = adj.withColumn('Market_Value', col('split_adj_price') * col('p_com_shs_out'))
    
    adj = adj.withColumn('year', F.year('p_date'))
    
    adj =adj.withColumn('week_of_year', F.weekofyear('p_date'))

    window_spec = Window.partitionBy('fsym_id', 'year', 'week_of_year').orderBy(col('p_date').desc())

    adj = adj.withColumn('row_num', F.row_number().over(window_spec))

    adj = adj.filter(col('row_num') == 1).orderBy('p_date')
    adj = adj.drop('temp_cum_split_factor', 'cum_split_factor', 'cum_split_factor_no_lag', 'row_num', 'p_split_date', 'p_split_factor')
    return adj

In [18]:
import csv


def consistent_changes(ticker, big_string):
    start_date = pd.to_datetime("2009-01-01")
    query1 = f"""
                SELECT b.FF_PRICE_CLOSE_FP, {big_string}
                FROM FF_ADVANCED_AF a
                LEFT JOIN sym_ticker_region d ON d.fsym_id = a.fsym_id
                LEFT JOIN FF_BASIC_QF b ON b.date=a.date AND d.fsym_id=b.fsym_id
                WHERE d.ticker_region = "{ticker}-US" 
                AND a.date >= "{start_date}"
                ORDER BY a.date
            """
    q_df = spark.sql(query1)
    print("query done")
    q_df = ps.DataFrame(q_df)
    non_string_columns = q_df.select_dtypes(exclude=['object']).columns
    q_df = q_df[non_string_columns]
    print("filtering done")
    # null_threshold = 10
    # columns_to_drop = q_df.columns[q_df.isnull().sum() > null_threshold]
    # q_df = q_df.drop(columns=columns_to_drop)
    correlations = q_df.corr()['FF_PRICE_CLOSE_FP']
    print("corr done")
    return correlations


    

def corr_query(implosion_df, col_string, table):
    df = get_fund_data(implosion_df)
    window_spec = Window.partitionBy('fsym_id', 'year').orderBy(col('p_date').desc())

    df = df.withColumn('row_num', F.row_number().over(window_spec))

    df = df.filter(col('row_num') == 1).orderBy('p_date') #should we compare correlations with market val?
    #should we do quarterly?
    
    df.createOrReplaceTempView("temp_table")
    query1 = f"""
                SELECT t.fsym_id, t.split_adj_price, t.Market_Value, t.p_date, {col_string}
                FROM temp_table t
                LEFT JOIN {table} a ON t.fsym_id = a.fsym_id AND YEAR(t.p_date)=YEAR(a.date)
                ORDER BY t.fsym_id, t.p_date
            """
 
    q_df = spark.sql(query1)
    window_spec = Window.partitionBy('fsym_id').orderBy('p_date')
    
    q_df = q_df.withColumn("return_market_val", (F.col('Market_Value') - F.lag('Market_Value').over(window_spec)) / F.lag('Market_Value').over(window_spec))
    q_df = q_df.withColumn("return", (F.col('split_adj_price') - F.lag('split_adj_price').over(window_spec)) / F.lag('split_adj_price').over(window_spec))
    
    return_columns = [c[2:] for c in col_string.split(", ")]
    mean_corrs = []

    for column in return_columns:
        return_col_name = f"return_{column}"
        corr_col_name = f"corr_with_{column}"
        q_df = q_df.withColumn(return_col_name, (F.col(column) - F.lag(column).over(window_spec)) / F.lag(column).over(window_spec))
        q_df = q_df.withColumn(corr_col_name, F.corr(return_col_name, 'return_market_val').over(window_spec))
        mean_corr_value = q_df.agg(F.mean(corr_col_name)).collect()[0][0]
        mean_corrs.append({'metric': column, 'mean_corr': mean_corr_value})
    return mean_corrs

def corr_analysis(table):
    imp_df_price = pd.read_csv('imploded_stocks_price.csv', index_col=False)
    imp_df_price = imp_df_price.loc[imp_df_price['Implosion_Start_Date'].notnull()]
    cols = get_not_null_cols(imp_df_price, 'FF_ADVANCED_DER_AF')
    result_string = ', '.join('a.' + item for item in cols)
    mean_corrs = corr_query(spark.createDataFrame(imp_df_price), result_string, 'FF_ADVANCED_DER_AF')
    mean_corrs_df = pd.DataFrame(mean_corrs)
    print(mean_corrs_df.head())
    mean_corrs_df = mean_corrs_df.sort_values(by='mean_corr', ascending=False)
    print(mean_corrs_df.head())

    
    
corr_analysis('FF_Advanced_Der_AF')

/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_list` loads all data into the driver's memory. It should only be used if the resulting list is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


+--------+----------+--------+------------+--------------+-------------+-----------+---------+-------------+
| fsym_id|    p_date| p_price|p_split_date|p_split_factor|p_divs_exdate|p_divs_s_pd|p_divs_pd|p_com_shs_out|
+--------+----------+--------+------------+--------------+-------------+-----------+---------+-------------+
|B0CXCV-R|2018-07-27|  4.6997|        null|          null|         null|       null|     null|      14764.0|
|B0CXCV-R|2018-07-30|  4.4126|        null|          null|         null|       null|     null|      14764.0|
|B0CXCV-R|2018-07-31|    4.75|        null|          null|         null|       null|     null|      14764.0|
|B0CXCV-R|2018-08-01|     4.5|        null|          null|         null|       null|     null|      14764.0|
|B0CXCV-R|2018-08-02|    4.71|        null|          null|         null|       null|     null|      14764.0|
|B0CXCV-R|2018-08-03|4.719999|        null|          null|         null|       null|     null|      14764.0|
|B0CXCV-R|2018-08-0

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
def get_cols():
    df_metrics = ps.DataFrame(spark.sql("SELECT * FROM FF_BASIC_AF LIMIT 10")) #get all the metrics
    cols = []
    for c in df_metrics.columns:
        if df_metrics[c].dtype=='float64':#get all the metrics we can calculate correlations with
            cols.append(c)
    return cols

#%change average of each feature plotted for pharmacy industry
def industry_analysis():
    stock_df = get_all_stocks_df()
    #stock_df = pd.read_csv('imploded_stocks.csv')
    #stock_df = spark.createDataFrame(stock_df)
    cols = ['ff_gross_inc', 'ff_sales', 'FF_OPER_EXP_TOT', 'FF_CASH_ST']
    col_string = ', '.join('a.' + item for item in cols)
    stock_df.createOrReplaceTempView("temp_table")
    q = f"""SELECT e.factset_industry_desc, t.ticker_region, a.date, {col_string} FROM temp_table t
    LEFT JOIN FF_BASIC_AF a ON a.fsym_id = t.fsym_id
    LEFT JOIN sym_coverage sc ON sc.fsym_id = t.fsym_id
    LEFT JOIN ff_sec_entity_hist c on c.fsym_id=sc.fsym_security_id
    LEFT JOIN sym_entity_sector d on d.factset_entity_id=c.factset_entity_id
    LEFT JOIN factset_industry_map e on e.factset_industry_code=d.industry_code
    WHERE a.date >= "2009-01-01" AND e.factset_industry_desc="Regional Banks"
    ORDER BY t.ticker_region,a.date"""
    ind_df = spark.sql(q)
    #print(ind_df.show(10))
    ind_df =ind_df.toPandas()
    ind_df['date'] = pd.to_datetime(ind_df['date'])
    new_cols = []
    for column in cols:
        ind_df[f'{column}_percentage_change'] = ind_df.groupby('ticker_region')[column].pct_change() * 100
        ind_df[f'{column}_percentage_change'].replace([np.inf, -np.inf], np.nan, inplace=True)
        ind_df.drop(column, axis=1, inplace=True)
        new_cols.append(f'{column}_percentage_change')
    ind_df['year'] = ind_df['date'].dt.year
    avg_pct_change = ind_df.groupby(['year'])[new_cols].mean().reset_index()
    print(avg_pct_change.head(20))
    num_rows = (len(new_cols) + 1) // 2  # Adjust the number of rows as needed
    num_cols = 2
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5 * num_rows))
    for i,column in enumerate(new_cols):
        row = i//num_cols
        col = i % num_cols 
        axes[row,col].plot(avg_pct_change['year'], avg_pct_change[column])
        axes[row, col].set_title(f'Avg {column} Percentage Change Over Time')
        axes[row, col].set_xlabel('Year')
        axes[row, col].set_ylabel(f'Avg {column} Percentage Change')
        axes[row, col].grid(True)
    plt.tight_layout()
    plt.show()

#industry_analysis()

In [ ]:
def get_top_bottom_five(df):
    df = df.sort_values(by='Value')
    df=df.dropna()
    top5 = df.head(10)
    down5 = df.tail(10)
    print(top5)
    print(down5)
    print(len(df))


# df = pd.read_csv('Avg_Correlations.csv', index_col=None)
# get_top_bottom_five(df)
#YOU'VE DONE WORST CHANGES NOW FIND OUT WHICH ONES DECREASE CONSISTENTLY
#ALSO FIGURE OUT MEANS BEFORE PERIOD AND AFTER PERIOD USING QUARTERLY AND COMPARE DIFF
#FINALLY WITH A HUGE LIST USE BORUTA

In [ ]:
metric_df

In [ ]:
metric_dict